In [1]:
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import time

In [15]:
import os 

__file__ = 'Untitled'
#현재 파일 이름
print(__file__)

#현재 파일 실제 경로
print(os.path.realpath(__file__))

#현재 파일 절대 경로
print(os.path.abspath(__file__))

Untitled
C:\Users\charl\OneDrive\Desktop\기타\coding\coding_on_study\First_project\code\Untitled
C:\Users\charl\OneDrive\Desktop\기타\coding\coding_on_study\First_project\code\Untitled


In [21]:
# CSV 파일 경로
input_path = "../jeju_final_2.csv"
output_path = "../jeju_with_coords.csv"

In [22]:
# 데이터 로드
df = pd.read_csv(input_path)

# 지오코더 설정 (user_agent는 구분 가능하게 임의 이름 사용)
geolocator = Nominatim(user_agent="jeju_mapper")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)


In [18]:
# 좌표 캐시 딕셔너리
coord_cache = {}

# 위경도 가져오기 함수 (캐싱 포함)

def get_lat_lon_cached(address):
    if pd.isna(address):
        return (None, None)
    if address in coord_cache:
        return coord_cache[address]
    try:
        location = geocode(address)
        if location:
            coord_cache[address] = (location.latitude, location.longitude)
        else:
            coord_cache[address] = (None, None)
    except Exception as e:
        print(f"Error at address: {address} -> {e}")
        coord_cache[address] = (None, None)
    return coord_cache[address]


In [23]:

# 진행률 확인
total = len(df)
start_time = time.time()

출발_lat = []
출발_lon = []
도착_lat = []
도착_lon = []

In [24]:
for idx, row in df.iterrows():
    출발 = row['출발 주소']
    도착 = row['도착 주소']

    s_lat, s_lon = get_lat_lon_cached(출발)
    d_lat, d_lon = get_lat_lon_cached(도착)

    출발_lat.append(float(s_lat))
    출발_lon.append(float(s_lon))
    도착_lat.append(float(d_lat))
    도착_lon.append(float(d_lon))

    # 진행 상태 출력
    if idx % 10 == 0:
        elapsed = time.time() - start_time
        print(f"[{idx}/{total}] 처리 중... 경과시간: {elapsed:.1f}초")

RateLimiter caught an error, retrying (0/2 tries). Called with (*('제주특별자치도 서귀포시 대륜동',), **{}).
Traceback (most recent call last):
  File "C:\anaconda\conda3\Lib\site-packages\urllib3\connectionpool.py", line 536, in _make_request
    response = conn.getresponse()
               ^^^^^^^^^^^^^^^^^^
  File "C:\anaconda\conda3\Lib\site-packages\urllib3\connection.py", line 507, in getresponse
    httplib_response = super().getresponse()
                       ^^^^^^^^^^^^^^^^^^^^^
  File "C:\anaconda\conda3\Lib\http\client.py", line 1428, in getresponse
    response.begin()
  File "C:\anaconda\conda3\Lib\http\client.py", line 331, in begin
    version, status, reason = self._read_status()
                              ^^^^^^^^^^^^^^^^^^^
  File "C:\anaconda\conda3\Lib\http\client.py", line 292, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\anaconda\conda3\Lib\socket.py", line 720, in readinto
    return

[0/4992] 처리 중... 경과시간: 24.3초


RateLimiter caught an error, retrying (0/2 tries). Called with (*('제주특별자치도 서귀포시 동홍동',), **{}).
Traceback (most recent call last):
  File "C:\anaconda\conda3\Lib\site-packages\urllib3\connectionpool.py", line 536, in _make_request
    response = conn.getresponse()
               ^^^^^^^^^^^^^^^^^^
  File "C:\anaconda\conda3\Lib\site-packages\urllib3\connection.py", line 507, in getresponse
    httplib_response = super().getresponse()
                       ^^^^^^^^^^^^^^^^^^^^^
  File "C:\anaconda\conda3\Lib\http\client.py", line 1428, in getresponse
    response.begin()
  File "C:\anaconda\conda3\Lib\http\client.py", line 331, in begin
    version, status, reason = self._read_status()
                              ^^^^^^^^^^^^^^^^^^^
  File "C:\anaconda\conda3\Lib\http\client.py", line 292, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\anaconda\conda3\Lib\socket.py", line 720, in readinto
    return

TypeError: float() argument must be a string or a real number, not 'NoneType'

In [ ]:
# 결과를 DataFrame에 추가
df['출발_lat'] = 출발_lat
df['출발_lon'] = 출발_lon
df['도착_lat'] = 도착_lat
df['도착_lon'] = 도착_lon

# ✅ 여기서 열 순서 재정렬
columns_order = [
    '출발행정코드', '도착행정코드',
    '출발 주소', '도착 주소',
    '출발_lat', '출발_lon',
    '도착_lat', '도착_lon'
]
other_columns = [col for col in df.columns if col not in columns_order]
df = df[columns_order + other_columns]

In [ ]:
# 결과 저장
df.to_csv(output_path, index=False)